In [10]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import csv
import datetime as dt
#file = open('GNNews.csv',"w")

In [11]:
def serchPage():
    
    count = 0     #計算抓多少
    titles = []   #標題
    contents = [] #標題文章
    links = []    #連結
    page = "https://gnn.gamer.com.tw/"
    driver = webdriver.Chrome('chromedriver')
    driver.get(page)
    thisMonthButton = driver.find_element_by_class_name("GN-lbox2G").find_element_by_tag_name("button")#按下本月新聞按鈕
    thisMonthButton.click()
    time.sleep(5)
    soup = bs(driver.page_source,"html.parser")
    driver.close()
    newsContainer = soup.find('div',{'class':'BH-lbox GN-lbox2'}) #抓連結
    for element in newsContainer.find_all('div',{'class':'GN-lbox2B'}):
        count+=1
        reader = element.find('h1',{'class':'GN-lbox2D'}).find('a',href = True)
        title = reader.text.strip()
        link = "https://"+reader.get('href')
        reader = element.find('p',{'class':'GN-lbox2C'})
        content = reader.text.strip()[0:-5]
        titles.append(title)
        contents.append(content)
        links.append(link)
        if(count>10):
            break;
    serchPagedf = pd.DataFrame({"標題":titles,"部分文章":contents,"連結":links})
    return serchPagedf

In [12]:
def newsPage(GNNews):
    count = 0
    GNNews['發表日期'] = 0
    GNNews['文章內容'] = ""
    texts = []
    dates = []
    pages = GNNews['連結']
    for page in pages:
        count+=1
        print(count)
        driver = webdriver.Chrome('chromedriver')
        driver.get(page)
        soup = bs(driver.page_source,"html.parser")
        news = soup.find('div',{'id':'BH-master'})
        if("BH-lbox GN-lbox3" in driver.page_source):
            text = news.find('div',{'class':'GN-lbox3B'}).text.strip()
            date = news.find('span',{'class':'GN-lbox3C'})
            if(date is None):
                date = news.find('span',{'class':'GN-lbox3CA'}).text.strip()
                i = date.find('）')+1
                date = date[i:i+11]
            else:
                date = date.text.strip()
                i = date.find('）')+1
                date = date[i:i+11]

        elif("BH-lbox MSG-list8"in driver.page_source):
            text = news.find('div',{'class':'MSG-list8C'}).text.strip()
            date = news.find('span',{'class':'ST1'}).text.strip()
            i = int(date.find('│')+1)
            while("2018" not in date[i:i+4]):
                i = date.find('│',i)+1
            date = date[i:i+10]
        texts.append(text)
        dates.append(date)
        driver.close()
        time.sleep(15)
    GNNews['發表日期'] = dates
    GNNews['文章內容'] = texts
    return GNNews

In [13]:
if __name__ == '__main__':
    GNNews = serchPage()
    time.sleep(5)
    GNNews = newsPage(GNNews)
    GNNews.to_csv('GNNews.csv',encoding ="utf_8_sig",index=False) #輸出成utf_8格式

1
2
3
4
5
6
7
8
9
10
11
